In [21]:
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [ ]:
import os
import pickle
from source.config import INTERIM_DATA_DIR, TESTING_DATA_DIR
from source.utils import sanitize_filename


def filter_points_near_road(
    df: pd.DataFrame, road_lon: float, road_lat: float, radius_km: float = 5
):
    """
    Filters points within a given radius (in km) from a single road location.

    Parameters:
        df (pd.DataFrame): DataFrame with 'Latitude' and 'Longitude' columns.
        road_lon (float): Longitude of the road.
        road_lat (float): Latitude of the road.
        radius_km (float): Radius in kilometers for filtering points.

    Returns:
        pd.DataFrame: Filtered DataFrame with points within the radius.
    """
    # Convert points and road coordinates to radians
    lat1, lon1 = np.radians(road_lat), np.radians(road_lon)
    lat2, lon2 = np.radians(df["Latitude"].values), np.radians(df["Longitude"].values)

    # Compute Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0) ** 2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distances = 6371 * c  # Earth's radius in km

    # Filter points within the radius
    df_filtered = df[distances <= radius_km].reset_index(drop=True)
    df_filtered = df_filtered[(df_filtered['Hastighet'] > 40) & (df_filtered['Hastighet'].notna()) & (df_filtered['Hastighet'].notnull())]
    return df_filtered

def load_polygon_boundary_from_file(road, subpath):
    file_path = os.path.join(
        INTERIM_DATA_DIR / "estimated_registrations",
        subpath,
        f"{sanitize_filename(road)}_boundary.pkl",
    )
    if os.path.exists(file_path):
        with open(file_path, "rb") as f:
            polygon_boundary = pickle.load(f).to_crs("EPSG:4326").geometry.iloc[0]
        return polygon_boundary
    else:
        file_path = os.path.join(
            INTERIM_DATA_DIR / "estimated_registrations",
            subpath,
            f"{road}_boundary.pkl",
        )
        if os.path.exists(file_path):
            with open(file_path, "rb") as f:
                polygon_boundary = pickle.load(f).to_crs("EPSG:4326").geometry.iloc[0]
            return polygon_boundary
        else:
            file_path = os.path.join(
                TESTING_DATA_DIR
                / "estimated_registrations"
                / f"{sanitize_filename(road)}_boundary.pkl",
            )
            if os.path.exists(file_path):
                with open(file_path, "rb") as f:
                    polygon_boundary = pickle.load(f).to_crs("EPSG:4326").geometry.iloc[0]
                return polygon_boundary
            else:
                raise FileNotFoundError(f"No polygon boundary file found for road: {file_path}")



2025-02-16 18:08:04.641 | INFO     | source.config:<module>:13 - PROJ_ROOT path is: /home/anders/engasjement_svv


In [12]:
df = pd.read_csv('../data/interim/estimated_registrations/processed-truck_only.csv')

In [13]:
dff = filter_points_near_road(df, 11.570661932248345, 60.88848554945865, 0.15)

In [16]:
polygon_boundary = load_polygon_boundary_from_file('tangensvingen_bru_tangensvingen_vest', 'bwim74t')

In [17]:
from shapely import Point


df_result = dff[dff.apply(
    lambda entry: polygon_boundary.contains(Point(entry["Longitude"], entry["Latitude"])), axis=1)
]


In [22]:
df_result

,VIN,Dato,Latitude,Longitude,Hastighet,Tonnage
0,YS2R6X40002170509,2021-07-14 07:43:07,60.887726,11.569538,58.000000,60
1,YS2R6X40002170509,2021-07-16 11:44:13,60.887821,11.569608,50.000000,60
2,YS2R6X40002175084,2021-08-23 05:52:44,60.888660,11.570980,65.000000,65
3,YS2R6X40002175084,2021-08-23 15:40:27,60.889202,11.571682,58.000000,65
4,YS2R6X40002170509,2021-08-26 05:31:49,60.887501,11.569177,58.000000,60
5,YS2R6X40002175084,2021-08-31 07:18:20,60.888474,11.570657,58.000000,65
6,YS2R6X40002175084,2021-09-06 09:44:36,60.889004,11.571459,61.000000,65
7,YS2R6X40002175084,2021-09-07 10:28:04,60.887840,11.569715,61.000000,65
8,YS2R6X40002175084,2021-09-09 07:04:35,60.887482,11.569052,29.000000,65
9,YS2R6X40002175084,2021-09-10 12:37:52,60.887886,11.569777,43.000000,65
